## Description

##### Implement a GAN from the following paper: https://arxiv.org/abs/1406.2661

### Libraries

In [ ]:
import tensorflow as tf
import numpy as np

### Data Setup

In [ ]:
x = tf.constant(np.array([[1, 2, 3, 4],
                         [1, 3, 5, 6],
                         [2, 5, 6, 7],
                         [3, 5, 7, 8]
                         ]), dtype=tf.float32)

In [ ]:
y = np.array([1, 1, 0, 1])

In [ ]:
x.shape

In [ ]:
x.shape[-1]

### Dense Layer

In [ ]:
class Dense(tf.Module):
    
    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        self.is_built = False # is built flag for dynamic input size inference
        self.out_features = out_features
        
    def __call__(self, x):
        if not self.is_built:
            self.w = tf.Variable(
                tf.random.normal([x.shape[-1], self.out_features]), name='w')
            self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
            self.is_built = True
        
        x_hat = tf.matmul(x, self.w) + self.b
        return x_hat

In [ ]:
dense_test = Dense(out_features=3)

In [ ]:
dense_test.__call__(x=x)

### Discriminator

##### Discriminator takes in as input, the output of the Generator that creates the "fake" image reconctruction from noisy data. Output of the discriminator is a probability prediction of 1 or 0 if image is from data (i.e. 1) or from the generator (i.e 0 and hence "fake") 

In [ ]:
class Discriminator(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        
        self.dense_1 = Dense(out_features=10000)
        self.dense_2 = Dense(out_features=1)
       
    def __call__(self, x):
        x = self.dense_1(x)
        x = tf.nn.relu(x)
        x = self.dense_2(x)
        return tf.nn.sigmoid(x)

In [ ]:
discr = Discriminator(name="discriminator")

In [ ]:
discr.__call__(x=x)

### Generator

##### Generator takes in random noise and outputs an image tensor which is same size as the real image

In [ ]:
flat_img = 4

In [ ]:
rnd_noise = tf.constant(np.array([[1, 2, 3, 4, 9],
                         [1, 3, 5, 6, 20],
                         [2, 5, 6, 7, 43],
                         [3, 5, 7, 8, 30]
                         ]), dtype=tf.float32)

In [ ]:
class Generator(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        
        self.dense_1 = Dense(out_features=10000)
        self.dense_2 = Dense(out_features=4000)
        self.dense_3 = Dense(out_features=flat_img)
        
    def __call__(self, x):
        x = self.dense_1(x)
        x = tf.nn.leaky_relu(x) # leaky_relu good for vanishing gradients
        x = self.dense_2(x)
        x = tf.nn.leaky_relu(x)
        return self.dense_3(x)

In [ ]:
gen = Generator(name="generator")

In [ ]:
gen(x=rnd_noise)

### Loss Function & Optimizers

In [ ]:
y_true_test = np.array([1, 0, 1, 1])
y_pred_test = np.array([1, 1, 0, 1])

In [ ]:
opt_d = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, name="discr_opt")
opt_g = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, name="opt_opt")

In [ ]:
def loss(y_true, y_pred):
    
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, 1)), dtype=tf.float32)
    y_pred_tf = tf.cast(tf.reshape(y_pred, (-1, 1)), dtype=tf.float32)
    
    return tf.compat.v1.losses.sigmoid_cross_entropy(y_true_tf, y_pred_tf)

In [ ]:
loss(y_true=y_true_test, y_pred=y_pred_test)

### Training

In [ ]:
epochs = 2
discr_epochs = 3
gen_epochs = 3

In [ ]:
for epoch in range(epochs):
    
    # discriminator training
    for k in range(discr_epochs):
        print(f"Discr Epoch nr is: {k}")
        
        with tf.GradientTape() as tape:
            pred_discr = discr(x=x)
            loss_discr = loss(y_true=y, y_pred=pred_discr)
        # do backprop
        grads = tape.gradient(loss_discr, discr.trainable_variables)
        opt_d.apply_gradients(zip(grads, discr.trainable_variables))
        print(f"Current discr loss from real input is {tf.reduce_mean(loss_discr)}")
        
        with tf.GradientTape() as tape:
            pred_gen = gen(x=rnd_noise)
            pred_discr_from_gen = discr(pred_gen)
            loss_gen = loss(y_true=y, y_pred=pred_discr_from_gen)
        # do backprop
        grads = tape.gradient(loss_gen, discr.trainable_variables)
        opt_d.apply_gradients(zip(grads, discr.trainable_variables))
        print(f"Current discr loss from fake input is {loss_gen}")
    
    # generator training
    for i in range(gen_epochs):
        print(f"Gen Epoch nr is: {i}")
              
        with tf.GradientTape() as tape:
            pred_gen = gen(x=rnd_noise)
            print(f"pred_gen is: {pred_gen}")
            print(f"pred discr is: {discr(pred_gen)}")
#             loss_gen = loss(y_true=x, y_pred=pred_gen) # use standard optimizer
            loss_gen = 1 - discr(pred_gen) # using loss in the paper instead
        # do backprop
        grads = tape.gradient(loss_gen, gen.trainable_variables)
        opt_g.apply_gradients(zip(grads, gen.trainable_variables))
        print(f"Current gen loss is {tf.reduce_mean(loss_gen)}")